In [5]:
pip! install deap

ERROR: unknown command "!"


In [11]:
import random
from deap import base, creator, tools, algorithms

n_nodos = 5
n_poblacion = 10
mutacion = 0.3

pesos = [
    [0, 7, 9, 8, 20],
    [7, 0, 10, 4, 11],
    [9, 10, 0, 15, 5],
    [8, 4, 15, 0, 17],
    [20, 11, 5, 17, 0]
]
names_nodos = ['A', 'B', 'C', 'D', 'E']

#Se crea un tipo de aptitud para problemas de minimización, se define un tipo 
#de individuo basado en una lista y se registran las funciones necesarias 
#para generar individuos y poblaciones en el toolbox

#Se crea un nuevo tipo de aptitud llamado "FitnessMin" utilizando la función create() del módulo creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# Se crea un nuevo tipo de individuo llamado "Individual"  que es una lista y se asocia con fitnessMin
creator.create("Individual", list, fitness=creator.FitnessMin)
# toolbox actúa como un contenedor para registrar las operaciones genéticas y los componentes del algoritmo genético.
toolbox = base.Toolbox()
toolbox.register("permutation", random.sample, range(n_nodos), n_nodos)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def distancia_nodos(individual):
    distancia = 0
    for i in range(n_nodos - 1):
        nodo_actual = individual[i]
        nodo_siguiente = individual[i + 1]
        distancia += pesos[nodo_actual][nodo_siguiente]
    return distancia,

def mutar(individual):
    if random.random() < mutacion:
        indices = random.sample(range(n_nodos), 2)
        individual[indices[0]], individual[indices[1]] = individual[indices[1]], individual[indices[0]]
    return individual,

#funciones de evaluación, cruce, mutación y selección en el objeto toolbox de DEAP. 

toolbox.register("evaluacion", distancia_nodos)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", mutar)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(42)

    poblacion = toolbox.population(n=n_poblacion)

    for gen in range(50):
        #Se aplica la operación genética varAnd (variación y cruce) a la población actual utilizando los operadores registrados en el toolbox
        offspring = algorithms.varAnd(poblacion, toolbox, cxpb=0.5, mutpb=0.2)
        # Se calcula la aptitud de cada individuo en la descendencia utilizando la función de evaluación registrada como "evaluate"
        fitnesses = toolbox.map(toolbox.evaluacion, offspring)
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit
        #Se seleccionan los mejores individuos de la descendencia utilizando la función de 
        #selección registrada como "select" en el toolbox. Se reemplaza la población actual por los individuos seleccionados.
        poblacion = toolbox.select(offspring, k=n_poblacion)

    best_individuo = tools.selBest(poblacion, k=1)[0]
    #Se crea una lista mejor_recorrido que contiene los nombres de los nodos correspondientes al mejor individuo.
    mejor_recorrido = [names_nodos[i] for i in best_individuo]

    print("Mejor recorrido:", mejor_recorrido)

if __name__ == "__main__":
    main()


Mejor recorrido: ['E', 'C', 'A', 'B', 'D']
